In [8]:
import xml.etree.ElementTree as ET

auto_topics = "/net/per610a/export/das18a/satoh-lab/share/datasets/trecvid/INS/topics/ins.auto.topics.2019.xml"
progress_topics = "/net/per610a/export/das18a/satoh-lab/share/datasets/trecvid/INS/topics/ins.progress.topics.2019.xml"
interactive_topics = "/net/per610a/export/das18a/satoh-lab/share/datasets/trecvid/INS/topics/ins.interactive.topics.2019.xml"
topics = [auto_topics, progress_topics, interactive_topics]

In [19]:
action_id_dict = {}
for topic in topics:
    root = ET.parse(topic).getroot()
    for child in root:
        if child.attrib['action'] in action_id_dict:
            action_id_dict[child.attrib['action']].append(int(child.attrib['num']))
        else:
            action_id_dict[child.attrib['action']] = [int(child.attrib['num'])]
            
topic_ids = []
for key in action_id_dict:
    topic_ids += action_id_dict[key]
    action_id_dict[key] = set(action_id_dict[key])

In [20]:
action_id_dict

{'holding_glass': {9249, 9250, 9251, 9252},
 'sit_on_couch': {9253, 9254, 9269, 9279, 9280},
 'holding_phone': {9255, 9256, 9257, 9281, 9282},
 'drinking': {9258, 9272, 9273, 9283, 9284},
 'open_door_enter': {9259, 9260, 9287, 9288},
 'shouting': {9261, 9262, 9274, 9289, 9290},
 'eating': {9263, 9264},
 'crying': {9265, 9275},
 'laughing': {9266, 9276},
 'open_door_leave': {9267, 9277, 9293, 9294},
 'go_up_down_stairs': {9268, 9278},
 'carrying_bag': {9270, 9271},
 'kissing': {9285, 9286},
 'hugging': {9291, 9292},
 'stand_talk_door': {9295, 9296},
 'close_door_wo_leaving': {9297, 9298}}

In [25]:
import csv

label_num = 1
action_label_num_list = []
action_id_num_list = []
for k, v in action_id_dict.items():
    action_label_num_list.append([k, label_num])
    action_id_num_list += [[topic_id, label_num] for topic_id in v]
    label_num += 1

# <ラベル名>    <ラベル番号>
# holding_glass      1
with open("../data/eastenders/action_labal_num.csv", "w") as fp:
    writer = csv.writer(fp, delimiter=' ')
    writer.writerows(action_label_num_list)

# <ラベルID> <ラベル番号>
#    9249         1
with open("../data/eastenders/action_id_num.csv", "w") as fp:
    writer = csv.writer(fp, delimiter=' ')
    writer.writerows(action_id_num_list)

In [30]:
with open("../data/eastenders/action_id_num.csv", "r") as fp:
    rows = csv.reader(fp, delimiter=' ')
    num_id_dict = {row[0]: row[1] for row in rows}
    
with open("/net/per610a/export/das18a/satoh-lab/share/datasets/trecvid/INS/ins.search.qrels.tv19") as f:
    rows = [row.split() for row in f.readlines()]
    gt_list = [[row[2], num_id_dict[row[0]]] for row in rows]

# <ビデオショット> <ラベル番号>
#   shot100_100         1
with open("../data/eastenders/grand_truth.csv", "w") as fp:
    writer = csv.writer(fp, delimiter=' ')
    writer.writerows(gt_list)

In [55]:
with open("../data/eastenders/grand_truth.csv", "r") as f:
    rows = csv.reader(f, delimiter=' ')
    gt_dict = {row[0]: row[1] for row in rows}
    
gt_32_list = []
gt_64_list = []
with open("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/videos.csv", "r") as fp:
    rows = csv.reader(fp, delimiter=' ')
    for row in rows:
        v = os.path.basename(row[0]).split('_')
        s = v[0] + "_" + v[1]
        gt_label = gt_dict[s] if s in gt_dict else 99
        if int(row[1]) < 64:
            gt_32_list.append([row[0], gt_label])
        else:
            gt_64_list.append([row[0], gt_label])

In [59]:
len(gt_dict)

79821

In [60]:
#  <ファイルパス>   <ラベル番号>
# shot1_1_id_1.mp4       1
with open("../data/eastenders/grand_truth_32.csv", "w") as fp:
    writer = csv.writer(fp, delimiter=' ')
    writer.writerows(gt_32_list)
    
with open("../data/eastenders/grand_truth_64.csv", "w") as fp:
    writer = csv.writer(fp, delimiter=' ')
    writer.writerows(gt_64_list)
    
#  <ファイルパス>   <ビデオインデックス>
# shot1_1_id_1.mp4           0
with open("../data/eastenders/grand_truth_32_index.csv", "w") as fp:
    gt_32_idx_list = [[gt_32_list[i][0], i] for i in range(len(gt_32_list))]
    writer = csv.writer(fp, delimiter=' ')
    writer.writerows(gt_32_idx_list)
    
with open("../data/eastenders/grand_truth_64_index.csv", "w") as fp:
    gt_64_idx_list = [[gt_64_list[i][0], i] for i in range(len(gt_64_list))]
    writer = csv.writer(fp, delimiter=' ')
    writer.writerows(gt_64_idx_list)

In [ ]:
import numpy as np

# 特徴ベクトルの整理
features_32 = np.load("../data/features/eastenders_32.npy")
labels_32 = np.load("../data/features/eastenders_32_labels.npy")
fvs_32 = np.array([np.mean(features[np.where(labels_32[:, 1] == idx)], axis=0) for idx in range(int(len(features_32) / 12))])

features_64 = np.load("../data/features/eastenders_64.npy")
labels_64 = np.load("../data/features/eastenders_64_labels.npy")
fvs_64 = np.array([np.mean(features[np.where(labels_64[:, 1] == idx)], axis=0) for idx in range(int(len(features_64) / 12))])

# 特徴ベクトルのリストを結合し保存
fvs = np.concatnate([fvs_32, fvs_64])
np.save("../data/eastenders/features", fvs)

# GT と映像のインデックスを紐付ける
with open("../data/eastenders/grand_truth.csv", "r") as f:
    rows = csv.reader(f, delimiter=' ')
    gt_dict = {row[0]: row[1] for row in rows}

idx = 0
gt_video_idx_list = []
with open("../data/eastenders/grand_truth_32_index.csv", "r") as fp:
    rows = csv.reader(fp, delimiter=' ')
    for row in rows:
        file_name = os.path.basename(row[0])[:-4]
        v = file_name.split('_')
        video_shot = v[0] + "_" + v[1]
        gt_label = gt_dict[s] if s in gt_dict else 99
        gt_video_idx_list.append([file_name, idx, gt_label])
        idx += 1
    
with open("../data/eastenders/grand_truth_64_index.csv", "w") as fp:
    rows = csv.reader(fp, delimiter=' ')
    for row in rows:
        file_name = os.path.basename(row[0])[:-4]
        v = file_name.split('_')
        video_shot = v[0] + "_" + v[1]
        gt_label = gt_dict[s] if s in gt_dict else 99
        gt_video_idx_list.append([file_name, idx, gt_label])
        idx += 1
        
with open("../data/eastenders/gt_video_index.csv", "w") as fp:
    writer = csv.writer(fp, delimiter=' ')
    writer.writerows(gt_video_idx_list)

In [67]:
with open("../data/eastenders/grand_truth_32_index.csv", "r") as fp:
    rows = csv.reader(fp, delimiter=' ')
    l = [r for r in rows]
    splited_list = [l[i::12] for i in range(12)]

cnt = 0    
for s in splited_list:
    with open("../data/eastenders/video_list_32/list_{}.csv".format(cnt), "w") as fp:
        writer = csv.writer(fp, delimiter=' ')
        writer.writerows(s)
    cnt += 1

In [68]:
with open("../data/eastenders/grand_truth_64_index.csv", "r") as fp:
    rows = csv.reader(fp, delimiter=' ')
    l = [r for r in rows]
    splited_list = [l[i::12] for i in range(12)]

cnt = 0    
for s in splited_list:
    with open("../data/eastenders/video_list_64/list_{}.csv".format(cnt), "w") as fp:
        writer = csv.writer(fp, delimiter=' ')
        writer.writerows(s)
    cnt += 1

In [ ]:
# original_video.csv を作成
# <id> <video_path> <action_gt_label>
# id : int
# video_path : str
# action_gt_label : int, optional

In [71]:
import glob

videos = glob.glob("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video/*.mp4")
videos[0]

'/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video/shot0_24.mp4'

In [72]:
"shot0_24.mp4"[:-4]

'shot0_24'

In [81]:
import os, re

In [75]:
original_videos = []
video_id = 1
for v in videos:
    shot = os.path.basename(v)[:-4]
    label = gt_dict[shot] if shot in gt_dict else '99'
    original_videos.append([video_id, v, label])
    video_id += 1

In [78]:
with open("../data/eastenders/original_videos.csv", "w") as fp:
    writer = csv.writer(fp, delimiter=' ')
    writer.writerows(original_videos)

In [80]:
original_video_dict = {os.path.basename(v[1])[:-4]: v[0] for v in original_videos}
original_video_dict

{'shot0_24': 1,
 'shot0_2': 2,
 'shot0_1': 3,
 'shot0_15': 4,
 'shot0_26': 5,
 'shot0_28': 6,
 'shot0_32': 7,
 'shot0_30': 8,
 'shot0_25': 9,
 'shot0_17': 10,
 'shot0_31': 11,
 'shot0_29': 12,
 'shot0_4': 13,
 'shot0_18': 14,
 'shot0_21': 15,
 'shot0_6': 16,
 'shot0_13': 17,
 'shot0_14': 18,
 'shot0_8': 19,
 'shot0_16': 20,
 'shot0_27': 21,
 'shot0_22': 22,
 'shot0_20': 23,
 'shot0_19': 24,
 'shot0_7': 25,
 'shot0_5': 26,
 'shot0_23': 27,
 'shot0_9': 28,
 'shot0_10': 29,
 'shot0_11': 30,
 'shot0_12': 31,
 'shot0_3': 32,
 'shot0_33': 33,
 'shot14_14': 34,
 'shot40_1024': 35,
 'shot7_203': 36,
 'shot38_1446': 37,
 'shot12_718': 38,
 'shot5_926': 39,
 'shot1_1698': 40,
 'shot8_1721': 41,
 'shot23_498': 42,
 'shot25_109': 43,
 'shot16_706': 44,
 'shot30_758': 45,
 'shot26_1717': 46,
 'shot10_1314': 47,
 'shot28_1156': 48,
 'shot45_337': 49,
 'shot3_1296': 50,
 'shot35_1212': 51,
 'shot32_98': 52,
 'shot18_685': 53,
 'shot33_2010': 54,
 'shot37_552': 55,
 'shot43_1495': 56,
 'shot55_1962': 

In [82]:
# person_videos.csv
# <id> <original_video_id> <person_video_path> <person_gt_label>
# id : int
# original_video_id : int -> original_videos.id
# person_video_path : str
# person_gt_label : int, optional

person_videos = []
person_video_id = 1

with open("../data/eastenders/grand_truth_32.csv", "r") as fp:
    reader = csv.reader(fp, delimiter=' ')
    for r in reader:
        video_path = r[0]
        video_file = re.split("[_.]", os.path.basename(video_path))
        video_shot = video_file[0] + "_" + video_file[1]
        original_video_id = original_video_dict[video_shot]
        person_videos.append([person_video_id, original_video_id, video_path, None])
        person_video_id += 1
        
with open("../data/eastenders/grand_truth_64.csv", "r") as fp:
    reader = csv.reader(fp, delimiter=' ')
    for r in reader:
        video_path = r[0]
        video_file = re.split("[_.]", os.path.basename(video_path))
        video_shot = video_file[0] + "_" + video_file[1]
        original_video_id = original_video_dict[video_shot]
        person_videos.append([person_video_id, original_video_id, video_path, None])
        person_video_id += 1

In [86]:
with open("../data/eastenders/person_videos.csv", "w") as fp:
    writer = csv.writer(fp, delimiter=' ')
    writer.writerows(person_videos)